In [ ]:
import pandas as pd
import sys

# Load the CSV file into a DataFrame
file_path = 'data.csv'
df = pd.read_csv(file_path)

# Convert 'Time' and 'Time Out' columns to datetime format
df['Time'] = pd.to_datetime(df['Time'])
df['Time Out'] = pd.to_datetime(df['Time Out'], errors='coerce', format='%m/%d/%Y %I:%M %p')

# Sort the DataFrame by 'Employee Name' and 'Time' to ensure correct calculations
df.sort_values(by=['Employee Name', 'Time'], inplace=True)

# Calculate time differences between consecutive shifts for each employee
df['TimeDiff'] = df.groupby('Employee Name')['Time'].diff().dt.total_seconds() / 3600

# Create a mask for employees who have worked for 7 consecutive days
consecutive_days_mask = df.groupby('Employee Name')['Time'].diff().dt.days.fillna(0).eq(1).rolling(7).sum() >= 7

# Get the names of employees who have worked for more than 7 consecutive days
result_names_7_consecutive_days = df.loc[consecutive_days_mask, 'Employee Name'].unique()

# Create masks for employees who meet the other criteria
time_between_shifts_mask = (df['TimeDiff'] > 1) & (df['TimeDiff'] < 10)
long_shift_mask = df['TimeDiff'] > 14

# Get the names of employees who meet the criteria
result_names_time_between_shifts = df.loc[time_between_shifts_mask, 'Employee Name'].unique()
result_names_long_shift = df.loc[long_shift_mask, 'Employee Name'].unique()

# Redirect the standard output to a file
sys.stdout = open("output.txt", "w")

# Display the names of employees who have worked for more than 7 consecutive days
print("\nNames of employees who have worked for more than 7 consecutive days:")
for name in result_names_7_consecutive_days:
    print(name)

print("---")

# Display the names of employees who have worked less than 10 hours between shifts but greater than 1 hour
print("Employees who have less than 10 hours between shifts but greater than 1 hour:")
for name in result_names_time_between_shifts:
    print(name)

print("---")

# Display the names of employees who have worked for more than 14 hours in a single shift
print("Employees who have worked for more than 14 hours in a single shift:")
for name in result_names_long_shift:
    print(name)

# Close the file to save changes
sys.stdout.close()
